# Scripts para crear los archivos txt con los inserts

Importanción de librerías:

In [3]:
import pandas as pd
import numpy as np
from itertools import chain
from database import Database

db1 = Database('../db/fifa25v1.db')
db2 = Database('../db/fifa25v2.db')

Ruta global y lectura de los datasets

In [4]:
BASE_PATH = './../docs/'
df_females = pd.read_csv(f'{BASE_PATH}female_players.csv', sep=',')
df_males = pd.read_csv(f'{BASE_PATH}male_players.csv', sep=',')
df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')
df_all.columns

/tmp/ipykernel_25358/3131349356.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(f'{BASE_PATH}all_players.csv', sep=',')


Index(['id', 'Rank', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY',
       'Acceleration', 'Sprint Speed', 'Positioning', 'Finishing',
       'Shot Power', 'Long Shots', 'Volleys', 'Penalties', 'Vision',
       'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing',
       'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control',
       'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness',
       'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength',
       'Aggression', 'Position', 'Weak foot', 'Skill moves', 'Preferred foot',
       'Height', 'Weight', 'Alternative positions', 'Age', 'Nation', 'League',
       'Team', 'play style', 'url', 'GK Diving', 'GK Handling', 'GK Kicking',
       'GK Positioning', 'GK Reflexes', 'Unnamed: 57'],
      dtype='object')

### Llenar Jugadores

Vamos a leer los <b>teams</b> y los vamos a colocar en una lista. Esta lista no deberá incluir repetidos. Generar un .txt con los inserts con el contenido de la lista creada para usarlos en esta tabla.

```sql
CREATE TABLE players (
  id	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, 
  rank INTEGER,
  name VARCHAR(50),
  overall INTEGER,
  velocity INTEGER,
  shooting INTEGER,
  passing INTEGER,
  dribbling INTEGER,
  defending INTEGER,
  physicality INTEGER,
  position VARCHAR(50),
  weak_foot INTEGER,
  skill_moves INTEGER,
  foot VARCHAR(50),
  height INTEGER,
  weight INTEGER,
  alternative_positions VARCHAR(50),
  age INTEGER,
  nation VARCHAR(50),
  league VARCHAR(50),
  team VARCHAR(50),
  play_styles VARCHAR(50),
  url VARCHAR(120),
  diving INTEGER,
  handling INTEGER,
  kicking INTEGER,
  positioning INTEGER,
  reflexes INTEGER,
  sex VARCHAR(10)
);
```

In [29]:
import math
value = float('nan')

male_playes_data = df_males[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2', 'Alternative positions', 'League', 'play style']].to_numpy()
female_playes_data = df_females[['Name', 'Rank', 'Weak foot', 'Skill moves', 'Height', 'Weight', 'url', 'Preferred foot', 'Position', 'Nation', 'Team', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Age', 'url2', 'Alternative positions', 'League', 'play style']].to_numpy()

text_players = '-- migrate:up \n\n'
id = 0
# fill males
for player in male_playes_data:
  # player data
  id = id + 1
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot = player[7]
  sex = 'Hombre'
  alternative_positions = 'NULL' if player[25] == 'nan' else f"'{player[25]}'"
  position = player[8]
  nation = player[9].replace("'", "''")
  team = player[10].replace("'", "''")
  age = player[23]
  league = player[26].replace("'", "''")
  styles = ''
  #print('1 ++++++++++++++++++++++++++++')
  #print(player[27])
  #print(type(player[27]))
  if isinstance(player[27], str):
    styles = ("'" + player[27] + "'") 
  else:
    styles = 'NULL'
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # position_id
  # nombre del pais de la liga
  rs = db1.fetchone(f"""
    SELECT NA.name AS nation
    FROM players PL 
    INNER JOIN teams TE ON PL.team_id = TE.id 
    INNER JOIN leagues LE ON TE.league_id = LE.id 
    INNER JOIN nations NA ON NA.id = LE.nation_id
    WHERE PL.id = {id}
  """)
  nation_league = rs['nation']
  tmp = f"INSERT INTO players (id, name, rank, weak_foot, skill_moves, height, weight, url, foot, sex, alternative_positions, position, nation, team, age, league, nation_league, styles, overall, velocity, shooting, passing, dribbling, defending, physicality, diving, handling, kicking, positioning, reflexes) VALUES (  {id}, '{name}', {rank}, {weak_foot}, {skill_moves}, {height}, {weight}, '{url}', '{foot}', '{sex}', {alternative_positions}, '{position}', '{nation}', '{team}', {age}, '{league}', '{nation_league}', {styles}, {overall}, {velocity}, {shooting}, {passing}, {dribbling}, {defending}, {physicality}, {diving}, {handling}, {kicking}, {positioning}, {reflexes});\n"
  tmp = tmp.replace("nan, nan, nan, nan, nan", "NULL, NULL, NULL, NULL, NULL")
  text_players += tmp
  #print(tmp)
  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# fill females
# fill males
for player in female_playes_data:
  # player data
  id = id + 1
  name = player[0].replace("'", "''")
  rank = player[1]
  weak_foot = player[2]
  skill_moves = player[3]
  height = player[4].split(' / ')[0][:-2]
  weight = player[5].split(' / ')[0][:-2]
  url = player[6] + player[24]
  foot = player[7]
  sex = 'Mujer'
  alternative_positions = 'NULL' if player[27] == 'nan' else f"'{player[27]}'"
  position = player[8]
  nation = player[9].replace("'", "''")
  team = player[10].replace("'", "''")
  age = player[23]
  league = player[26].replace("'", "''")
  styles = ''
  if isinstance(player[27], str):
    styles = ("'" + player[27] + "'") 
  else:
    styles = 'NULL'
  # common data
  overall = player[11]
  velocity = player[12] # PAC = velocidad
  shooting = player[13]
  passing = player[14]
  dribbling = player[15]
  defending = player[16]
  physicality = player[17]
  # gk data
  diving = player[18]
  handling = player[19]
  kicking = player[20]
  positioning = player[21]
  reflexes = player[22]
  # nombre del pais de la liga
  rs = db1.fetchone(f"""
    SELECT NA.name AS nation
    FROM players PL 
    INNER JOIN teams TE ON PL.team_id = TE.id 
    INNER JOIN leagues LE ON TE.league_id = LE.id 
    INNER JOIN nations NA ON NA.id = LE.nation_id
    WHERE PL.id = {id}
  """)
  nation_league = rs['nation']
  tmp = f"INSERT INTO players (id, name, rank, weak_foot, skill_moves, height, weight, url, foot, sex, alternative_positions, position, nation, team, age, league, nation_league, styles, overall, velocity, shooting, passing, dribbling, defending, physicality, diving, handling, kicking, positioning, reflexes) VALUES (  {id}, '{name}', {rank}, {weak_foot}, {skill_moves}, {height}, {weight}, '{url}', '{foot}', '{sex}', {alternative_positions}, '{position}', '{nation}', '{team}', {age}, '{league}', '{nation_league}', {styles}, {overall}, {velocity}, {shooting}, {passing}, {dribbling}, {defending}, {physicality}, {diving}, {handling}, {kicking}, {positioning}, {reflexes});\n"
  tmp = tmp.replace("nan, nan, nan, nan, nan", "NULL, NULL, NULL, NULL, NULL")
  text_players += tmp

  # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# write male an female
text_players = text_players + "UPDATE players SET alternative_positions = NULL WHERE alternative_positions = 'nan';\n-- migrate:down \n\nDELETE FROM players;"
with open(f'{BASE_PATH}players2_inserts.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text_players)